In [15]:
def load_conll_dataset(filepath):
    '''Reads in a conllx file; generates Observation objects
    
    For each sentence in a conllx file, generates a single Observation
    object.

    Args:
      filepath: the filesystem path to the conll dataset
  
    Returns:
      A list of Observations 
    '''
    observations = []
    lines = (x for x in open(filepath))
    for buf in generate_lines_for_sent(lines):
        conllx_lines = []
        for line in buf:
            conllx_lines.append(line.strip().split('\t'))
        embeddings = [None for x in range(len(conllx_lines))]
        #observation = self.observation_class(*zip(*conllx_lines), embeddings)
        #observations.append(observation)
    return conllx_lines
    #return observations

def generate_lines_for_sent(lines):
    '''Yields batches of lines describing a sentence in conllx.

    Args:
      lines: Each line of a conllx file.
    Yields:
      a list of lines describing a single sentence in conllx.
    '''
    buf = []
    for line in lines:
        if line.startswith('#'):
            continue
        if not line.strip():
            if buf:
                yield buf
                buf = []
            else:
                continue
        else:
            buf.append(line.strip())
    print(buf)
    if buf:
        print(count)
        yield buf

In [16]:
lines = load_conll_dataset("/users/ujan/linguistic-structures/data/raw/Universal Dependencies 2.11/ud-treebanks-v2.11/UD_English-LinES/en_lines-ud-train.conllu")

[]


In [14]:
len(lines)

12

In [1]:
import datasets
from datasets import load_dataset

In [4]:
en_treebanks = ['en_esl', 'en_ewt', 'en_gum', 'en_gumreddit', 'en_lines', 'en_partut', 'en_pronouns', 'en_pud']

In [5]:
ud_en0 = load_dataset('universal_dependencies', en_treebanks[0])
ud_en1 = load_dataset('universal_dependencies', en_treebanks[1])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4124 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /users/ujan/.cache/huggingface/datasets/universal_dependencies/en_esl/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/12543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2077 [00:00<?, ? examples/s]

Dataset universal_dependencies downloaded and prepared to /users/ujan/.cache/huggingface/datasets/universal_dependencies/en_ewt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
ud_en1

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 12543
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 2002
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 2077
    })
})

In [16]:
ud_en1['train'][13]

{'idx': 'weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-0014',
 'text': 'In Suwayrah, Kut Province, two car bombs were discovered before they could be detonated.',
 'tokens': ['In',
  'Suwayrah',
  ',',
  'Kut',
  'Province',
  ',',
  'two',
  'car',
  'bombs',
  'were',
  'discovered',
  'before',
  'they',
  'could',
  'be',
  'detonated',
  '.'],
 'lemmas': ['in',
  'Suwayrah',
  ',',
  'Kut',
  'Province',
  ',',
  'two',
  'car',
  'bombs',
  'be',
  'discover',
  'before',
  'they',
  'could',
  'be',
  'detonate',
  '.'],
 'upos': [2, 10, 1, 10, 10, 1, 3, 0, 0, 17, 16, 5, 11, 17, 17, 16, 1],
 'xpos': ['IN',
  'NNP',
  ',',
  'NNP',
  'NNP',
  ',',
  'CD',
  'NN',
  'NN',
  'VBD',
  'VBN',
  'IN',
  'PRP',
  'MD',
  'VB',
  'VBN',
  '.'],
 'feats': ['None',
  "{'Number': 'Sing'}",
  'None',
  "{'Number': 'Sing'}",
  "{'Number': 'Sing'}",
  'None',
  "{'NumType': 'Card'}",
  "{'Number': 'Sing'}",
  "{'Number': 'Sing'}",
  "{'Mood': 'Ind', 'Tense': 'Past', 'VerbFor

In [6]:
ud_en_lines['train'][2]

{'idx': 'en_lines-ud-train-doc1-3',
 'text': 'Some of the content in this topic may not be applicable to some languages.',
 'tokens': ['Some',
  'of',
  'the',
  'content',
  'in',
  'this',
  'topic',
  'may',
  'not',
  'be',
  'applicable',
  'to',
  'some',
  'languages',
  '.'],
 'lemmas': ['some',
  'of',
  'the',
  'content',
  'in',
  'this',
  'topic',
  'may',
  'not',
  'be',
  'applicable',
  'to',
  'some',
  'language',
  '.'],
 'upos': [11, 2, 8, 0, 2, 8, 0, 17, 7, 17, 6, 2, 8, 0, 1],
 'xpos': ['IND',
  None,
  'DEF',
  'SG-NOM',
  None,
  'DEM-SG',
  'SG-NOM',
  'PRES-AUX',
  'NEG',
  'INF',
  'POS',
  None,
  'IND',
  'PL-NOM',
  'Period'],
 'feats': ["{'Case': 'Nom'}",
  'None',
  "{'Definite': 'Def', 'PronType': 'Art'}",
  "{'Number': 'Sing'}",
  'None',
  "{'Number': 'Sing', 'PronType': 'Dem'}",
  "{'Number': 'Sing'}",
  "{'VerbForm': 'Fin'}",
  'None',
  "{'VerbForm': 'Inf'}",
  "{'Degree': 'Pos'}",
  'None',
  'None',
  "{'Number': 'Plur'}",
  'None'],
 'head': ['